In [54]:
# Step 1: Import the necessary modules

from sqlalchemy import create_engine, Column, Integer, String, Enum


In [29]:
# Step 2: Establish a database connection

engine = create_engine('sqlite:///./data/nobel_prize.db', echo=True)

#echo argument set to True allow to see the SQL instructions generated by SQL‐ Alchemy from the command line:


In [30]:
# Setp 3 create a Base class using declarative_base. This base will be used to create table classes, 
#from which SQLAlchemy will create the database’s table schemas

from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [31]:
# Step 4: Defining an SQL database table
from sqlalchemy import Column, Integer, String, Enum

class Winner(Base):
    __tablename__ = 'winners'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)
    year = Column(Integer)
    nationality = Column(String)
    sex = Column(Enum('male', 'female'))

    def __repr__(self):
        return "<Winner(name='%s', category='%s', year='%s')>" \
            % (self.name, self.category, self.year)

In [32]:
# Step 5: Create the database tables
Base.metadata.create_all(engine)

2024-09-08 12:17:12,326 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 12:17:12,328 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("winners")
2024-09-08 12:17:12,328 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-08 12:17:12,329 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("winners")
2024-09-08 12:17:12,330 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-08 12:17:12,330 INFO sqlalchemy.engine.Engine 
CREATE TABLE winners (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	category VARCHAR, 
	year INTEGER, 
	nationality VARCHAR, 
	sex VARCHAR(6), 
	PRIMARY KEY (id)
)


2024-09-08 12:17:12,331 INFO sqlalchemy.engine.Engine [no key 0.00037s] ()
2024-09-08 12:17:12,342 INFO sqlalchemy.engine.Engine COMMIT


In [33]:
# Step 6 Add Instances with a Session to interact with the DB
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [35]:
#Our target Data
nobel_winners = [
    {'category': 'Physics', 'name': 'Albert Einstein', 'nationality': 'Swiss', 'sex': 'male', 'year': 1921},
    {'category': 'Physics', 'name': 'Paul Dirac', 'nationality': 'British', 'sex': 'male', 'year': 1933},
    {'category': 'Chemistry', 'name': 'Marie Curie', 'nationality': 'Polish', 'sex': 'female', 'year': 1911}
]

In [36]:
#step 7 Use the Winner class to create instances and table rows and add them to the session:
albert = Winner(**nobel_winners[0])
session.add(albert)
session.new

IdentitySet([<Winner(name='Albert Einstein', category='Physics', year='1921')>])

In [37]:
#  We can remove the object using expunge
session.expunge(albert)
session.new

IdentitySet([])

In [38]:
# step 8 Add all the members of our nobel_winners list to the session and commit them to the database:
winner_rows = [Winner(**w) for w in nobel_winners]
session.add_all(winner_rows)
session.commit()

2024-09-08 12:25:11,043 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 12:25:11,045 INFO sqlalchemy.engine.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2024-09-08 12:25:11,046 INFO sqlalchemy.engine.Engine [generated in 0.00097s] ('Albert Einstein', 'Physics', 1921, 'Swiss', 'male')
2024-09-08 12:25:11,048 INFO sqlalchemy.engine.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2024-09-08 12:25:11,049 INFO sqlalchemy.engine.Engine [cached since 0.00355s ago] ('Paul Dirac', 'Physics', 1933, 'British', 'male')
2024-09-08 12:25:11,050 INFO sqlalchemy.engine.Engine INSERT INTO winners (name, category, year, nationality, sex) VALUES (?, ?, ?, ?, ?)
2024-09-08 12:25:11,050 INFO sqlalchemy.engine.Engine [cached since 0.00476s ago] ('Marie Curie', 'Chemistry', 1911, 'Polish', 'female')
2024-09-08 12:25:11,051 INFO sqlalchemy.engine.Engine COMMIT


In [39]:
# Step 9Querying the Database: 
#1) count the number of rows in our winners’ table:
session.query(Winner).count()

2024-09-08 12:29:34,662 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 12:29:34,665 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners) AS anon_1
2024-09-08 12:29:34,666 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()


3

In [40]:
## Step 9 Querying the Database: 
# 2) retrieve all Swiss winners
result = session.query(Winner).filter_by(nationality='Swiss')
list(result)

2024-09-08 12:30:34,523 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.nationality = ?
2024-09-08 12:30:34,523 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ('Swiss',)


[<Winner(name='Albert Einstein', category='Physics', year='1921')>]

In [41]:
## Step 9 Querying the Database: 
# 3) Get all non-Swiss Physics winners
result = session.query(Winner).filter(\
Winner.category == 'Physics', Winner.nationality != 'Swiss')
list(result)

2024-09-08 12:31:41,747 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.category = ? AND winners.nationality != ?
2024-09-08 12:31:41,747 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ('Physics', 'Swiss')


[<Winner(name='Paul Dirac', category='Physics', year='1933')>]

In [42]:
## Step 9 Querying the Database: 
# 4) Get a row based on ID number
session.query(Winner).get(3)

2024-09-08 12:33:02,580 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2024-09-08 12:33:02,581 INFO sqlalchemy.engine.Engine [generated in 0.00134s] (3,)


<Winner(name='Marie Curie', category='Chemistry', year='1911')>

In [43]:
## Step 9 Querying the Database:
# Retrieve winners ordered by year:
res = session.query(Winner).order_by('year')
list(res)

2024-09-08 12:33:43,509 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners ORDER BY winners.year
2024-09-08 12:33:43,510 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()


[<Winner(name='Marie Curie', category='Chemistry', year='1911')>,
 <Winner(name='Albert Einstein', category='Physics', year='1921')>,
 <Winner(name='Paul Dirac', category='Physics', year='1933')>]

In [46]:
#Step 10: Converts an SQLAlchemy instance to a dict
def inst_to_dict(inst, delete_id=True):
    dat = {}
    for column in inst.__table__.columns:
        dat[column.name] = getattr(inst, column.name)
    if delete_id:
        dat.pop('id')
    return dat

In [47]:
# Step 11 Reconstruct our nobel_winners target list
winner_rows = session.query(Winner)
nobel_winners = [inst_to_dict(w) for w in winner_rows]
nobel_winners

2024-09-08 12:37:27,090 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2024-09-08 12:37:27,091 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()


[{'name': 'Albert Einstein',
  'category': 'Physics',
  'year': 1921,
  'nationality': 'Swiss',
  'sex': 'male'},
 {'name': 'Paul Dirac',
  'category': 'Physics',
  'year': 1933,
  'nationality': 'British',
  'sex': 'male'},
 {'name': 'Marie Curie',
  'category': 'Chemistry',
  'year': 1911,
  'nationality': 'Polish',
  'sex': 'female'}]

In [48]:
# Step 12: Update database rows
marie = session.query(Winner).get(3)
marie.nationality = 'French'
session.dirty

IdentitySet([<Winner(name='Marie Curie', category='Chemistry', year='1911')>])

In [49]:
# Commit marie’s changes and check that her nationality has changed from Polish to French
session.commit()

2024-09-08 12:39:34,515 INFO sqlalchemy.engine.Engine UPDATE winners SET nationality=? WHERE winners.id = ?
2024-09-08 12:39:34,516 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ('French', 3)
2024-09-08 12:39:34,517 INFO sqlalchemy.engine.Engine COMMIT


In [50]:
session.dirty

IdentitySet([])

In [51]:
session.query(Winner).get(3).nationality

2024-09-08 12:40:12,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-08 12:40:12,036 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners 
WHERE winners.id = ?
2024-09-08 12:40:12,037 INFO sqlalchemy.engine.Engine [cached since 429.5s ago] (3,)


'French'

In [52]:
# Delete the results of a query
session.query(Winner).filter_by(name='Albert Einstein').delete()

2024-09-08 12:40:59,534 INFO sqlalchemy.engine.Engine DELETE FROM winners WHERE winners.name = ?
2024-09-08 12:40:59,535 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ('Albert Einstein',)


1

In [53]:
list(session.query(Winner))

2024-09-08 12:41:14,465 INFO sqlalchemy.engine.Engine SELECT winners.id AS winners_id, winners.name AS winners_name, winners.category AS winners_category, winners.year AS winners_year, winners.nationality AS winners_nationality, winners.sex AS winners_sex 
FROM winners
2024-09-08 12:41:14,466 INFO sqlalchemy.engine.Engine [cached since 227.4s ago] ()


[<Winner(name='Paul Dirac', category='Physics', year='1933')>,
 <Winner(name='Marie Curie', category='Chemistry', year='1911')>]

In [ ]:
# Drop the whole table by Winner.__table__.drop(engine)